<a href="https://colab.research.google.com/github/anweshaban21/finmodel-lab/blob/main/IndividualModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Individual Models for Bullish Regime

In [3]:
!pip install yfinance arch statsmodels matplotlib pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 18.9 MB/s eta 0:00:00


# BETA MODEL

In [15]:
def recommend_stocks_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']



            avg_return = np.mean(stock_returns)
            #sharpe_ratio = avg_return / predicted_vol if predicted_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'Avg_Return': round(avg_return, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()


    #print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = df['Beta']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'Avg_Return', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f},  Beta: {row['Beta']}")

    return top

In [16]:
recommend_stocks_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

⚠️ Not enough data for ZOMATO.NS
        Stock   Beta  Avg_Return  Risk_Score
 SUNPHARMA.NS 0.3288      0.0003      0.3288
BAJAJ-AUTO.NS 0.4164      0.0023      0.4164
ASIANPAINT.NS 0.4218      0.0008      0.4218
    MARUTI.NS 0.4261      0.0011      0.4261
       ITC.NS 0.4793      0.0028      0.4793
ULTRACEMCO.NS 0.5033      0.0014      0.5033
        LT.NS 0.7232      0.0012      0.7232
     TITAN.NS 0.7233      0.0014      0.7233
     WIPRO.NS 0.8984     -0.0002      0.8984
       TCS.NS 0.9475     -0.0000      0.9475
 ICICIBANK.NS 0.9933      0.0003      0.9933
  HDFCBANK.NS 1.2220      0.0003      1.2220
      INFY.NS 1.3211     -0.0013      1.3211
  RELIANCE.NS 1.3811     -0.0002      1.3811

✅ Top 5 Safest Stocks (Lowest Risk Score):
SUNPHARMA.NS → Risk Score: 0.3288,  Beta: 0.3288
BAJAJ-AUTO.NS → Risk Score: 0.4164,  Beta: 0.4164
ASIANPAINT.NS → Risk Score: 0.4218,  Beta: 0.4218
MARUTI.NS → Risk Score: 0.4261,  Beta: 0.4261
ITC.NS → Risk Score: 0.4793,  Beta: 0.4793


,Stock,Beta,Avg_Return,Risk_Score
7,SUNPHARMA.NS,0.3288,0.0003,0.3288
11,BAJAJ-AUTO.NS,0.4164,0.0023,0.4164
5,ASIANPAINT.NS,0.4218,0.0008,0.4218
6,MARUTI.NS,0.4261,0.0011,0.4261
8,ITC.NS,0.4793,0.0028,0.4793


# GARCH MODEL

In [17]:
def recommend_stocks_garch(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.linear_model import LinearRegression

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']



            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_volatility = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # scale back

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / garch_volatility if garch_volatility != 0 else 0

            results.append({
                'Stock': stock,
                'GARCH_Volatility': round(garch_volatility, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['GARCH_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()





    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = df['GARCH_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock','GARCH_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Vol: {row['GARCH_Volatility']}")

    return top

In [18]:
recommend_stocks_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

⚠️ Not enough data for ZOMATO.NS
        Stock  GARCH_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
 ICICIBANK.NS            0.0095      0.0003        0.0333      0.0095
    MARUTI.NS            0.0096      0.0011        0.1097      0.0096
 SUNPHARMA.NS            0.0097      0.0003        0.0275      0.0097
       TCS.NS            0.0102     -0.0000       -0.0036      0.0102
     WIPRO.NS            0.0102     -0.0002       -0.0239      0.0102
ULTRACEMCO.NS            0.0103      0.0014        0.1325      0.0103
ASIANPAINT.NS            0.0104      0.0008        0.0808      0.0104
  RELIANCE.NS            0.0104     -0.0002       -0.0145      0.0104
       ITC.NS            0.0104      0.0028        0.2642      0.0104
  HDFCBANK.NS            0.0105      0.0003        0.0318      0.0105
     TITAN.NS            0.0107      0.0014        0.1318      0.0107
        LT.NS            0.0109      0.0012        0.1123      0.0109
BAJAJ-AUTO.NS            0.0113      0.0023        0.2037

,Stock,GARCH_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
3,ICICIBANK.NS,0.0095,0.0003,0.0333,0.0095
6,MARUTI.NS,0.0096,0.0011,0.1097,0.0096
7,SUNPHARMA.NS,0.0097,0.0003,0.0275,0.0097
1,TCS.NS,0.0102,-0.0000,-0.0036,0.0102
13,WIPRO.NS,0.0102,-0.0002,-0.0239,0.0102


# LSTM MODEL

In [21]:
def recommend_stocks_lstm(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']


            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            #avg_vol = np.mean([lstm_vol])
            sharpe_ratio = avg_return / lstm_vol if lstm_vol != 0 else 0

            results.append({
                'Stock': stock,
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()


    # --- Risk Score ---
    df['Risk_Score'] = df['LSTM_Volatility']


    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, LSTM Vol: {row['LSTM_Volatility']}")

    return top


In [22]:
recommend_stocks_lstm(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When

⚠️ Not enough data for ZOMATO.NS
        Stock  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
    MARUTI.NS           0.0016      0.0011        0.6722      0.0016
     TITAN.NS           0.0016      0.0014        0.8995      0.0016
  RELIANCE.NS           0.0018     -0.0002       -0.0836      0.0018
       TCS.NS           0.0019     -0.0000       -0.0196      0.0019
       ITC.NS           0.0019      0.0028        1.4175      0.0019
 ICICIBANK.NS           0.0019      0.0003        0.1713      0.0019
 SUNPHARMA.NS           0.0020      0.0003        0.1322      0.0020
     WIPRO.NS           0.0020     -0.0002       -0.1234      0.0020
  HDFCBANK.NS           0.0021      0.0003        0.1580      0.0021
ASIANPAINT.NS           0.0023      0.0008        0.3644      0.0023
BAJAJ-AUTO.NS           0.0023      0.0023        1.0069      0.0023
ULTRACEMCO.NS           0.0024      0.0014        0.5646      0.0024
        LT.NS           0.0026      0.0012        0.4706      0.0026
 

,Stock,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
6,MARUTI.NS,0.0016,0.0011,0.6722,0.0016
12,TITAN.NS,0.0016,0.0014,0.8995,0.0016
4,RELIANCE.NS,0.0018,-0.0002,-0.0836,0.0018
1,TCS.NS,0.0019,-0.0000,-0.0196,0.0019
8,ITC.NS,0.0019,0.0028,1.4175,0.0019
